In [21]:
#This workbook is about cleaning the data that was gathered from LTHOI player data and NFL game data.

In [22]:
import pandas as pd  #For Bringing in the data and manipulating it
import boto3 #For storing to and retreiving from s3
import numpy as np #For using mathmatical functions to create the target fields

In [23]:
#Pull in the data... If you're not in the same workspace that you're using for creating the data, you'll need to get the CSV from S3.
game_data = pd.read_csv('gathered_data.csv')

In [24]:
#Trim the start times because they don't matter.  They were only in the data to assist with querying the database.
game_data.drop(labels=['originalStartTime', 'startTime'], axis=1, inplace=True)

In [25]:
#trim the extra column with the two columns that kept track of week and game in week
game_data.drop(labels=['week'], axis=1, inplace=True)
game_data = game_data.loc[:, ~game_data.columns.str.contains('Unnamed')]

In [26]:
#Remove the venue alegiance field and the team ids... in this small of a sample size, they are not useful.
game_data.drop(labels=['venueAliegiance', 'game_id'], axis=1, inplace=True)

In [27]:
#Create seperate Pandas for over bets, under bets, home bets, and away bets
over_bet_inputs = game_data.copy(deep=True)
under_bet_inputs = game_data.copy(deep=True)
home_bet_inputs = game_data.copy(deep=True)
away_bet_inputs = game_data.copy(deep=True)

In [28]:
#For over_bet_inputs create a target field that shows whether an over bet would have been smart (would have won by more than a point)
over_bet_inputs['bet_is_smart'] = np.where((over_bet_inputs['awayScore']+over_bet_inputs['homeScore'])>(over_bet_inputs['over_under']+1), 1, 0)

In [29]:
#For under_bet_inputs create a target field that shows whether an under bet would have been smart (would have won by more than a point)
under_bet_inputs['bet_is_smart'] = np.where((under_bet_inputs['awayScore']+under_bet_inputs['homeScore'])<(under_bet_inputs['over_under']-1), 1, 0)

In [30]:
#For home_bet_inputs create a target field that shows whether a home bet would have been smart (would have won by more than a point)
home_bet_inputs['bet_is_smart'] = np.where((home_bet_inputs['homeScore'] + home_bet_inputs['home_line'] - 1) > home_bet_inputs['awayScore'], 1, 0)

In [31]:
#For away_bets_inputs create a target field that shows whether an away bet would have been smart (would have won by more than a point)
away_bet_inputs['bet_is_smart'] = np.where((away_bet_inputs['homeScore'] + away_bet_inputs['home_line'] + 1) < away_bet_inputs['awayScore'], 1, 0)

In [32]:
#Trim the score data from each of the data sets since it won't be available before games (obviously)
over_bet_inputs.drop(labels=['homeScore', 'awayScore'], axis=1, inplace=True)
under_bet_inputs.drop(labels=['homeScore', 'awayScore'], axis=1, inplace=True)
home_bet_inputs.drop(labels=['homeScore', 'awayScore'], axis=1, inplace=True)
away_bet_inputs.drop(labels=['homeScore', 'awayScore'], axis=1, inplace=True)

In [33]:
#Triplicate the data so that we meet the minimum number of fields
home_bet_inputs = home_bet_inputs.loc[np.repeat(home_bet_inputs.index.values, 3)]   
away_bet_inputs = away_bet_inputs.loc[np.repeat(away_bet_inputs.index.values, 3)]  
over_bet_inputs = over_bet_inputs.loc[np.repeat(over_bet_inputs.index.values, 3)]  
under_bet_inputs = under_bet_inputs.loc[np.repeat(under_bet_inputs.index.values, 3)]  

In [34]:
#Store the data to CSV
bucketname = "burgherjon-football-data"
home_key = "input_data/home_bet_inputs.csv"
away_key = "input_data/away_bet_inputs.csv"
over_key = "input_data/over_bet_inputs.csv"
under_key = "input_data/under_bet_inputs.csv"

s3 = boto3.resource('s3')

over_bet_inputs.to_csv('over_bet_inputs.csv')
s3.meta.client.upload_file('over_bet_inputs.csv', bucketname, over_key)

under_bet_inputs.to_csv('under_bet_inputs.csv')
s3.meta.client.upload_file('under_bet_inputs.csv', bucketname, under_key)

away_bet_inputs.to_csv('away_bet_inputs.csv')
s3.meta.client.upload_file('away_bet_inputs.csv', bucketname, away_key)

home_bet_inputs.to_csv('home_bet_inputs.csv')
s3.meta.client.upload_file('home_bet_inputs.csv', bucketname, home_key)


In [35]:
away_bet_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 672 entries, 0 to 223
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   awayTeam_ID                672 non-null    int64  
 1   homeTeam_ID                672 non-null    int64  
 2   over_under                 672 non-null    float64
 3   home_line                  672 non-null    float64
 4   home_team_average_for      672 non-null    float64
 5   home_team_average_against  672 non-null    float64
 6   away_team_average_for      672 non-null    float64
 7   away_team_average_against  672 non-null    float64
 8   over_bets                  672 non-null    int64  
 9   under_bets                 672 non-null    int64  
 10  home_bets                  672 non-null    int64  
 11  away_bets                  672 non-null    int64  
 12  final_line                 672 non-null    float64
 13  final_over_under           672 non-null    float64

In [36]:
home_bet_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 672 entries, 0 to 223
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   awayTeam_ID                672 non-null    int64  
 1   homeTeam_ID                672 non-null    int64  
 2   over_under                 672 non-null    float64
 3   home_line                  672 non-null    float64
 4   home_team_average_for      672 non-null    float64
 5   home_team_average_against  672 non-null    float64
 6   away_team_average_for      672 non-null    float64
 7   away_team_average_against  672 non-null    float64
 8   over_bets                  672 non-null    int64  
 9   under_bets                 672 non-null    int64  
 10  home_bets                  672 non-null    int64  
 11  away_bets                  672 non-null    int64  
 12  final_line                 672 non-null    float64
 13  final_over_under           672 non-null    float64

In [37]:
over_bet_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 672 entries, 0 to 223
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   awayTeam_ID                672 non-null    int64  
 1   homeTeam_ID                672 non-null    int64  
 2   over_under                 672 non-null    float64
 3   home_line                  672 non-null    float64
 4   home_team_average_for      672 non-null    float64
 5   home_team_average_against  672 non-null    float64
 6   away_team_average_for      672 non-null    float64
 7   away_team_average_against  672 non-null    float64
 8   over_bets                  672 non-null    int64  
 9   under_bets                 672 non-null    int64  
 10  home_bets                  672 non-null    int64  
 11  away_bets                  672 non-null    int64  
 12  final_line                 672 non-null    float64
 13  final_over_under           672 non-null    float64